In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time

BASE_URL = "https://pl.wikipedia.org"
START_URL = "https://pl.wikipedia.org/wiki/Krak%C3%B3w"
HEADERS = {"User-Agent": "Mozilla/5.0"}

visited = set()
scraped_data = {}

def is_valid_link(href):
    return href.startswith("/wiki/") and not any(x in href for x in [":", "#"])

def scrape_page(url):
    try:
        print(f"Pobieram: {url}")
        response = requests.get(url, headers=HEADERS)
        soup = BeautifulSoup(response.text, "html.parser")
        content = soup.find("div", class_="mw-parser-output")
        if content:
            return content.get_text(separator="\n", strip=True)
    except Exception as e:
        print(f"Błąd przy {url}: {e}")
    return ""

# 1. Scrapuj stronę główną Krakowa
main_text = scrape_page(START_URL)
scraped_data[START_URL] = main_text
visited.add(START_URL)

# 2. Znajdź linki z pierwszego poziomu
response = requests.get(START_URL, headers=HEADERS)
soup = BeautifulSoup(response.text, "html.parser")
internal_links = {
    urljoin(BASE_URL, a['href'])
    for a in soup.find_all("a", href=True)
    if is_valid_link(a['href'])
}

# 3. Scrapuj każdy link (bez wchodzenia w dalsze linki)
for link in internal_links:
    if link not in visited:
        visited.add(link)
        page_text = scrape_page(link)
        scraped_data[link] = page_text
        time.sleep(0.5)  # aby nie przeciążać serwera

# 4. Zapisz dane do pliku
with open("krakow_wiki_scraped.txt", "w", encoding="utf-8") as f:
    for url, content in scraped_data.items():
        f.write(f"URL: {url}\n")
        f.write(content)
        f.write("\n" + "="*80 + "\n")


Pobieram: https://pl.wikipedia.org/wiki/Krak%C3%B3w
Pobieram: https://pl.wikipedia.org/wiki/Pradawne_i_pierwotne_lasy_bukowe_Karpat_i_innych_region%C3%B3w_Europy
Pobieram: https://pl.wikipedia.org/wiki/Misteria_Paschalia
Pobieram: https://pl.wikipedia.org/wiki/Harz
Pobieram: https://pl.wikipedia.org/wiki/Droga_krajowa_nr_79_(Polska)
Pobieram: https://pl.wikipedia.org/wiki/Wodorow%C4%99glany
Pobieram: https://pl.wikipedia.org/wiki/Tadeusz_Seweryn
Pobieram: https://pl.wikipedia.org/wiki/Krok_(w%C5%82adca_Czech)
Pobieram: https://pl.wikipedia.org/wiki/M%C5%9Bcis%C5%82aw_(Bia%C5%82oru%C5%9B)
Pobieram: https://pl.wikipedia.org/wiki/Osterode_am_Harz
Pobieram: https://pl.wikipedia.org/wiki/Encyklopedia_Britannica
Pobieram: https://pl.wikipedia.org/wiki/Trzebiat%C3%B3w
Pobieram: https://pl.wikipedia.org/wiki/Ryglice
Pobieram: https://pl.wikipedia.org/wiki/Dania
Pobieram: https://pl.wikipedia.org/wiki/Okr%C4%99g_Pomorze_Zachodnie
Pobieram: https://pl.wikipedia.org/wiki/Dom_W%C5%82oski_w_Krakowi